# Story 1

## Quest 1

### Part 1

In [202]:
import re

with open('inputs/everybody_codes_e1_q01_p1.txt') as f:
    s = f.read().split('\n')


def parse(eq):
    return tuple(int(x) for x in re.findall('\\d+', eq))


def eni(n, exp, mod):
    l = []
    val = 1
    for _ in range(exp):
        val = (val*n) % mod
        l.append(str(val))
    return int(''.join(l[::-1]))


def calculate(eq):
    (a,b,c,x,y,z,m) = parse(eq)
    return eni(a,x,m) + eni(b,y,m) + eni(c,z,m)


max([calculate(eq) for eq in s])

2279023507

### Part 2

In [203]:
with open('inputs/everybody_codes_e1_q01_p2.txt') as f:
    s = f.read().split('\n')


def memoize_all(n, exp, mod):
    cache = {}
    exp_cache = {}
    cycles = {}
    
    val = n
    e = 1
    while (val, n, mod) not in cache:
        val_new = (val*n) % mod
        exp_cache[(n,e,mod)] = val
        cache[(val, n, mod)] = (val_new, e)
        val = val_new
        e += 1
    cycle_start = cache[(val, n, mod)][1]
    cycle_length = e - cycle_start
    cycles[(n, mod)] = (cycle_start, cycle_length)

    return (exp_cache, cycles)


def value_at(n, exp, mod, exp_cache, cycles):
    (cycle_start, cycle_length) = cycles[(n, mod)]
    adjusted_exp = exp if exp < cycle_start else ((exp - cycle_start) % cycle_length) + cycle_start
    return exp_cache[(n, adjusted_exp, mod)]


def eni(n, exp, mod):
    (exp_cache, cycles) = memoize_all(n,exp,mod)
    
    l = []
    for i in range(5):
        l.append(str(value_at(n, exp-i, mod, exp_cache, cycles)))
    return int(''.join(l))


def calculate(eq):
    (a,b,c,x,y,z,m) = parse(eq)
    return eni(a,x,m) + eni(b,y,m) + eni(c,z,m)


max([calculate(eq) for eq in s])

105631190755985

### Part 3

In [204]:
with open('inputs/everybody_codes_e1_q01_p3.txt') as f:
    s = f.read().split('\n')


def offset_sum(n, exp, mod, exp_cache, cycles):
    (cycle_start, _) = cycles[(n, mod)]
    return sum([value_at(n, e, mod, exp_cache, cycles) for e in range(1,cycle_start)])


def cycle_sum(n, exp, mod, exp_cache, cycles):
    (cycle_start, cycle_length) = cycles[(n, mod)]
    return sum([value_at(n, e, mod, exp_cache, cycles) for e in range(cycle_start, cycle_start+cycle_length)])


def eni(n, exp, mod):
    (exp_cache, cycles) = memoize_all(n,exp,mod)
    
    (cycle_start, cycle_length) = cycles[(n, mod)]
    cycle_count = (exp-cycle_start) // cycle_length
    next_cycle_start = cycle_start + cycle_count*cycle_length
    offset_acc = offset_sum(n,exp,mod,exp_cache,cycles)
    full_cycles_acc = cycle_count*cycle_sum(n,exp,mod,exp_cache,cycles) 
    remaining_cycle_acc = sum([value_at(n,e,mod,exp_cache,cycles) for e in range(next_cycle_start,exp+1)])
    return offset_acc + full_cycles_acc + remaining_cycle_acc



max([calculate(eq) for eq in s])

602895845035823

## Quest 2

### Part 1

In [354]:
with open('inputs/everybody_codes_e1_q02_p1.txt') as f:
    s = f.read().split('\n')


def parse(line):
    tree_data = line.split(' ')[2:]
    tree_data = [td.split('[')[1][:-1].split(',') for td in tree_data]
    tree_data = tuple((int(td[0]), td[1]) for td in tree_data)
    return tree_data


def add_leaf(tree,current,rank,symbol):
    if current is None:
        tree['root'] = (rank,symbol)

    else:
        if rank < current[0]:
            if tree[current][0] is None:
                tree[current] = ((rank, symbol), tree[current][1])
            else:
                add_leaf(tree,tree[current][0],rank,symbol)
        else:
            if tree[current][1] is None:
                tree[current] = (tree[current][0], (rank, symbol))
            else:
                add_leaf(tree,tree[current][1],rank,symbol)
    
    if (rank,symbol) not in tree:
        tree[(rank,symbol)] = (None, None)
    return tree


def add(tree,rank,symbol):
    return add_leaf(tree,tree.get('root'),rank,symbol)


def add_to_trees(tree_l, tree_r, line):
    ((rank_l, sym_l), (rank_r, sym_r)) = parse(line)
    add(tree_l, rank_l, sym_l)
    add(tree_r, rank_r, sym_r)


def build(tree_data):
    tree_l = {}
    tree_r = {}
    for line in tree_data:
        add_to_trees(tree_l, tree_r, line)
    return (tree_l, tree_r)


def count_nodes_per_level(tree, current, current_level, counter):
    if current is not None:
        counter[current_level] = counter.get(current_level,0)+1

        count_nodes_per_level(tree, tree[current][0], current_level+1, counter)
        count_nodes_per_level(tree, tree[current][1], current_level+1, counter)

    return counter
    
def count_tree_nodes_per_level(tree):
    return count_nodes_per_level(tree, tree.get('root'), 0, {})


def read_node(tree, level, current, current_level):
    if current is None:
        return ''
    
    if current_level == level:
        return current[1]

    return read_node(tree, level, tree[current][0], current_level+1) + read_node(tree, level, tree[current][1], current_level+1)


def read_tree(tree, level):
    return read_node(tree, level, tree.get('root'), 0)


def read_level(tree_l, tree_r, level):
    return read_tree(tree_l, level) + read_tree(tree_r, level)


(tree_l, tree_r) = build(s)

level_sizes_l = count_tree_nodes_per_level(tree_l)
level_sizes_r = count_tree_nodes_per_level(tree_r)
max_level_l = max(level_sizes_l, key=level_sizes_l.get)
max_level_r = max(level_sizes_r, key=level_sizes_r.get)

read_tree(tree_l, max_level_l) + read_tree(tree_r, max_level_r)

'QUACK!BTHFNNJR'

### Part 2

In [20]:
with open('inputs/everybody_codes_e1_q02_p2.txt') as f:
    s = f.read().split('\n')


def parse(line):
    command = line.split(' ')[0]
    if command == 'SWAP':
        pair_id = int(line.split(' ')[1])
        return (command, pair_id)
    else:
        pair_id = int(line.split(' ')[1][3:])
        tree_data = line.split(' ')[2:]
        tree_data = [td.split('[')[1][:-1].split(',') for td in tree_data]
        tree_data = [(int(td[0]), td[1]) for td in tree_data]
        return (command, pair_id, tree_data[0], tree_data[1])


def add_leaf(tree,current_id,pair_id,nodes):
    (rank,symbol) = nodes[pair_id]
    
    if current_id is None:
        tree['root'] = pair_id
    
    else:
        if rank < nodes[current_id][0]:
            if tree[current_id][0] is None:
                tree[current_id] = (pair_id, tree[current_id][1])
            else:
                add_leaf(tree,tree[current_id][0],pair_id, nodes)
        else:
            if tree[current_id][1] is None:
                tree[current_id] = (tree[current_id][0], pair_id)
            else:
                add_leaf(tree,tree[current_id][1],pair_id, nodes)
    
    if (rank,symbol) not in tree:
        tree[pair_id] = (None, None)
    return tree


def add(tree,pair_id,rank,symbol,nodes):
    nodes[pair_id] = (rank, symbol)
    return add_leaf(tree,tree.get('root'),pair_id,nodes)


def swap(nodes_l, nodes_r, pair_id):
    tmp = nodes_l[pair_id]
    nodes_l[pair_id] = nodes_r[pair_id]
    nodes_r[pair_id] = tmp
    

def execute(tree_l, tree_r, nodes_l, nodes_r, line):
    cmd = parse(line)
    if cmd[0] == 'ADD':
        (_, pair_id, (rank_l, sym_l), (rank_r, sym_r)) = cmd
        add(tree_l, pair_id, rank_l, sym_l, nodes_l)
        add(tree_r, pair_id, rank_r, sym_r, nodes_r)
    else:
        (_, pair_id) = cmd
        swap(nodes_l, nodes_r, pair_id)


def build(commands):
    tree_l = {}
    tree_r = {}
    nodes_l = {}
    nodes_r = {}
    for line in commands:
        execute(tree_l, tree_r, nodes_l, nodes_r, line)
    return (tree_l, tree_r, nodes_l, nodes_r)


def count_nodes_per_level(tree, current, current_level, counter):
    if current is not None:
        counter[current_level] = counter.get(current_level,0)+1

        count_nodes_per_level(tree, tree[current][0], current_level+1, counter)
        count_nodes_per_level(tree, tree[current][1], current_level+1, counter)

    return counter


def count_tree_nodes_per_level(tree):
    return count_nodes_per_level(tree, tree.get('root'), 0, {})


def read_node(tree, level, current, current_level, nodes):
    if current is None:
        return ''
    
    if current_level == level:
        return nodes[current][1]

    return read_node(tree, level, tree[current][0], current_level+1, nodes) + read_node(tree, level, tree[current][1], current_level+1, nodes)


def read_tree(tree, nodes, level):
    return read_node(tree, level, tree.get('root'), 0, nodes)


(tree_l, tree_r, nodes_l, nodes_r) = build(s)

level_sizes_l = count_tree_nodes_per_level(tree_l)
level_sizes_r = count_tree_nodes_per_level(tree_r)
max_level_l = max(level_sizes_l, key=level_sizes_l.get)
max_level_r = max(level_sizes_r, key=level_sizes_r.get)

read_tree(tree_l, nodes_l, max_level_l) + read_tree(tree_r, nodes_r, max_level_r)

'QUACK!WRFMSGYVBPLHYS'

### Part 3

In [32]:
with open('inputs/everybody_codes_e1_q02_p3.txt') as f:
    s = f.read().split('\n')


def parse(line):
    command = line.split(' ')[0]
    if command == 'SWAP':
        pair_id = line.split(' ')[1]
        return (command, pair_id)
    else:
        pair_id = line.split(' ')[1][3:]
        tree_data = line.split(' ')[2:]
        tree_data = [td.split('[')[1][:-1].split(',') for td in tree_data]
        tree_data = [(int(td[0]), td[1]) for td in tree_data]
        return (command, pair_id, tree_data[0], tree_data[1])


def add_leaf(tree,tree_id,current_id,pair_id,nodes):
    (rank,symbol) = nodes[pair_id+tree_id]
    
    if current_id is None:
        tree['root'+tree_id] = pair_id+tree_id
    
    else:
        if rank < nodes[current_id][0]:
            if tree[current_id][0] is None:
                tree[current_id] = (pair_id+tree_id, tree[current_id][1])
            else:
                add_leaf(tree,tree_id,tree[current_id][0],pair_id, nodes)
        else:
            if tree[current_id][1] is None:
                tree[current_id] = (tree[current_id][0], pair_id+tree_id)
            else:
                add_leaf(tree,tree_id,tree[current_id][1],pair_id, nodes)
    
    if (rank,symbol) not in tree:
        tree[pair_id+tree_id] = (None, None)
    return tree


def add(tree,tree_id,pair_id,rank,symbol,nodes):
    nodes[pair_id+tree_id] = (rank, symbol)
    return add_leaf(tree,tree_id,tree.get('root'+tree_id),pair_id,nodes)


def swap(tree, nodes, pair_id):
    for node in tree:
        if node[:-1] == 'root':
            if tree[node] == pair_id+'l':
                tree[node] = pair_id+'r'
            elif tree[node] == pair_id+'r':
                tree[node] = pair_id+'l'
            
        else:
            (child_l, child_r) = tree[node]
        
            if child_l == pair_id+'l':
                tree[node] = (pair_id+'r', child_r)
            elif child_l == pair_id+'r':
                tree[node] = (pair_id+'l', child_r)
                
            if child_r == pair_id+'l':
                tree[node] = (child_l, pair_id+'r')
            elif child_r == pair_id+'r':
                tree[node] = (child_l, pair_id+'l')


def execute(tree, nodes, line):
    cmd = parse(line)
    if cmd[0] == 'ADD':
        (_, pair_id, (rank_l, sym_l), (rank_r, sym_r)) = cmd
        add(tree, 'l', pair_id, rank_l, sym_l, nodes)
        add(tree, 'r', pair_id, rank_r, sym_r, nodes)
    else:
        (_, pair_id) = cmd
        swap(tree, nodes, pair_id)


def build(commands):
    tree = {}
    nodes = {}
    for line in commands:
        execute(tree, nodes, line)
    return (tree, nodes)


def count_nodes_per_level(tree, current, current_level, counter):
    if current is not None:
        counter[current_level] = counter.get(current_level,0)+1

        count_nodes_per_level(tree, tree[current][0], current_level+1, counter)
        count_nodes_per_level(tree, tree[current][1], current_level+1, counter)

    return counter


def count_tree_nodes_per_level(tree, tree_id):
    return count_nodes_per_level(tree, tree.get('root'+tree_id), 0, {})


def read_node(tree, tree_id, level, current, current_level, nodes):
    if current is None:
        return ''
    
    if current_level == level:
        return nodes[current][1]

    return read_node(tree, tree_id, level, tree[current][0], current_level+1, nodes) + \
        read_node(tree, tree_id, level, tree[current][1], current_level+1, nodes)


def read_tree(tree, tree_id, nodes, level):
    return read_node(tree, tree_id, level, tree.get('root'+tree_id), 0, nodes)


(tree, nodes) = build(s)

level_sizes_l = count_tree_nodes_per_level(tree,'l')
level_sizes_r = count_tree_nodes_per_level(tree,'r')
max_level_l = max(level_sizes_l, key=level_sizes_l.get)
max_level_r = max(level_sizes_r, key=level_sizes_r.get)

read_tree(tree, 'l', nodes, max_level_l) + read_tree(tree, 'r', nodes, max_level_r)

'QUACK!BWBBNFGJBHTVBNHVGMVGWZGYNXWF'